# Multiclass Classification
    => One vs One
    => One vs Rest

In [ ]:
# One vs One -> useful for Large dataset which can't fit in memory
    # n-C-2 Classifiers, O(n^2) time

In [ ]:
# One vs Rest :-
    -> More memory
    -> N classifiers
    -> Parallel
    -> Less Time : o(n)
    -> common => Scikit learn ['ovr'] # default

## Dataset Preparation

In [3]:
import numpy as np
import os
from pathlib import Path
from keras.preprocessingprocessing import image # solve this issue too

In [ ]:
p = Path("../../Datasets/Images")# use your path and dataselts

dirs = p.glob("*")
labels_dict = ["cat":0,"dog":1,"horse":2,"human":3]

image_data = []
labels = []

for folder_dir in dirs:
    #print(folder_name)
    label = str(folder_dir).split("\\")[-1][-1]#slicing
    
    #print(label)
    for img_path in folder.glob("*.jpg"):
        #print(img_path)
        img = image.load_image(img_path,target_size=(32,32))
        img_array = image.img_to_array(img)
        image_data.append(img_array)
        labels.append(labels_dict[label])
    
print(len(image_data))
print(len(labels))
print(image_data.shape,labels.shape)
    
        

## Visualise this data!

In [ ]:
# Convert this into Numpy array
image_data = np.array(image_data,dtype='float32')/255.0
labels = np.array(labels)


# Randomly Shuffle the data
import random

# Zip
combined = list(zip(image_data,labels))
# Shuffle
random.shuffle(combined)
# Unzip
image_data[:],labels[:] = zip(*combined)

In [ ]:
def drawImg(img):
    from matplotlib import pyplot as plt
    plt.imshow(img)
    plt.axis("off")
    plt.show()
    return

for i in range(10):
    drawImg(image_data[i])

# SVM Classifier

In [ ]:
class SVM:
    
    def __init__(self,C=1.0):
        self.C = C
        self.W = 0
        self.b = 0
        
    def hingeLoss(self,W,b,X,Y):
        loss = 0.0
        loss += 0.5*np.dot(W,W.T)
        
        m = X.shape[0]
        
        for i in range(m):
            ti = Y[i]*(np.dot(W,X[i].T)+b)
            loss += self.C*max(0,(1-ti))
        
        return loss[0][0]
    
    def fit(self,X,Y,batch_size=100,learning_rate=0.001,maxItr=300):
        
        no_of_features = X.shape[1]
        no_of_samples = X.shape[0]
        
        n = learning_rate
        c = self.C
        
        # Init the model parameters
        
        W = np.zeros((1,no_of_features))
        bias = 0
        
        # Initial Loss
        print(self.hingeLoss(W,bias,X,Y))
        
        # Training from here....
        # Weight ans Bias update rule , earlier discussed....
        
        losses = []
        
        for i in range(maxItr):
            #Training Loop
            
            l = self.hingeLoss(W,bias,X,Y)
            losses.append(l)
            ids = np.arange(no_of_samples)
            np.random.shuffle(ids)
            # Batch Gradient Descent(paper) with random shuffling
            for batch_start in range(0,no_of_samples,batch_size):
                # Assume 0 gradient for the batch
                gradw = 0
                gradb = 0
                
                # Iterate over all examples in the mini batch
                for j in range(batch_start,batch_start+batch_size):
                    if j<no_of_samples:
                        i = ids[j]
                        ti = Y[i]*(np.dot(W,X[i].T)+bias)
                        if ti>1:
                            gradw += 0
                            gradb += 0
                        else:
                            gradw += c*Y[i]*X[i]
                            gradb += c*Y[i]
                            
                # Gradient for the batch is ready ! Update W,B
                W = W -n*W + n*gradw
                bias = bias + n*gradb
                
        self.W = W
        self.b = bias
        return W,bias,losses

# We need to convert data for One-for-One classification!

In [ ]:
print(image_data.shape)
print(labels.shape)
M = image_data.shape[0]
image_data = image_data.reshape(M,-1)
print(image_data.shape)
print(labels.shape)

In [ ]:
CLASSES = len(np.unique(labels))
print(CLASSES)

In [ ]:
def classWiseData(X,y):
    data = {}
    for i in range(CLASSES):
        data[i]=[]
    
    for i in range(X.shape[0]):
        data[y[i]].append(X[i])
    
    for k in data.keys():
        data[k] = np.array(data[k])
        
    return data
        
data = classWiseData(image_data,labels)

In [ ]:
print(data[0].shape[0])
print(data[1].shape[0])
print(data[2].shape[0])
print(data[3].shape[0])

In [5]:
def getDataPairForSVM(d1,d2):
    """ Combines Data of two classes into a single matrix """
    
    l1,l2 = d1.shape[0],d2.shape[0]
    
    samples = l1+l2
    features = d1.shape[1]
    
    data_pair = np.zeros((samples,features))
    data_labels = np.zeros((samples,))
    
    data_pair[:l1,:] =d1
    data_pair[:l2,:] =d2
    
    data_labels[:l1] = -1
    data_labels[:l2] = +1
    
    return data_pair,data_labels

## Training NC2 SVM's Part!

In [ ]:
mySVM = SVM()
xp,yp = getDataPairForSVM(data[0],data[1])
w,b,loss = mySVM.fit(xp,yp,learning_rate=0.00001,maxItr=1000)
print(loss)
plt.plot(loss)
plt.show()

In [ ]:
def trainSVMs(x,y):
    
    svm_classfiers = {}
    for i in range(10):
        svm_classfiers[i] = {}
        for j in range(i+1,CLASSES):
            xpair,ypair = getDataPairForSVM(data[i],data[j])
            wts,b,loss = mySVM.fit(xpair,ypair,learning_rate=0.00001,maxItr=1000)
            svm_classfiers[i][j] = {wts,b}
            # plotting loss
            plt.plot(loss)
            plt.show()
        
    return svm_classifiers
        

In [ ]:
trainSVMs(image_data,labels)

In [ ]:
# Parameters for Cats and dogs
cats_dogs = svm_classifiers[0][1]
cats_humans = svm_classifiers[0][3]
print(cats_dogs[0])
print(cats_dogs[1])

# Prediction

In [ ]:
def binaryPredict(x,w,b):
    z = np.dot(x,w.T)+b
    if z>=0:
        return 1;
    else:
        return -1

In [ ]:
def predict(x):
    
    count = np.zeros((CLASSES,))
    
    for i in range(CLASSES):
        for j in range(i+1,CLASSES):
            w,v = svm_classifiers[i][j]
            # Take a majority prediction
            z = binaryPredict(x,w,b)
            
            if (z==1):
                count[j] += 1
            else:
                count[i] += 1
                
    print(count)
    final_prediction = np.argmax(count)
    return final_prediction
            

In [ ]:
predict(image_data[0])
print(labels[0])

In [ ]:
def accuracy(x,y):
    count = 0
    for i in range(x.shape[0]):
        prediction = predict(x[i])
        if(prediction==y[i]):
            count+=1
            
    return count/x.shape[0]

In [ ]:
accuracy(image_data,labels)

# Prediction using sklearn classifier

In [ ]:
from sklearn import svm

In [ ]:
svm_classifier = svm.SVM(kernel='linear',C=1.0)

In [ ]:
svm_classifier.fit(image_data,labels)
svm_classifier.score(image_data,labels)